In [2]:
# Import Library

import pickle
import json
import pandas as pd
import numpy as np
import joblib
import os

In [3]:
# load the Files

#model dan preprocessing
with open('best_xgb.pkl', 'rb') as file_1:
  best_xgb= joblib.load(file_1)
  
with open('Drop_Columns.txt', 'r') as file_2:
 Drop_Columns = json.load(file_2)

with open('list_num_cols.txt', 'r') as file_3:
  num_columns = json.load(file_3)

with open('list_cat_columns.txt', 'r') as file_4:
  cat_columns = json.load(file_4)

with open('final_pipeline.pkl', 'rb') as file_5:
  final_pipeline = pickle.load(file_5)

In [4]:
# Menginput list data yang digunakan dari direktori
print(os.listdir("data"))

['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'Home_credit.ipynb', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv', 'Table_concate.ipynb']


In [5]:
POS_CASH_balance = pd.read_csv('data/POS_CASH_balance.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')
previous_application = pd.read_csv('data/previous_application.csv')
installments_payments = pd.read_csv('data/installments_payments.csv')
credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
bureau = pd.read_csv('data/bureau.csv')
application_test = pd.read_csv('data/application_test.csv')

In [6]:
data_inf =  application_test
data_inf.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


In [7]:
### POS_CASH_balance
# Menghapus nilai yang sama pada kolom 'SK_ID_CURR'
POS_CASH_balance = POS_CASH_balance.drop_duplicates(subset=['SK_ID_CURR'])

### previous_application
# Menghapus nilai yang sama pada kolom 'SK_ID_CURR'
previous_application = previous_application.drop_duplicates(subset=['SK_ID_CURR'])

### installments_payments
# Menghapus nilai yang sama pada kolom 'SK_ID_CURR'
installments_payments = installments_payments.drop_duplicates(subset=['SK_ID_CURR'])

### credit_card_balance
# Menghapus nilai yang sama pada kolom 'SK_ID_CURR'
credit_card_balance = credit_card_balance.drop_duplicates(subset=['SK_ID_CURR'])


### bureau
# Menghapus nilai yang sama pada kolom 'SK_ID_CURR'
bureau = bureau.drop_duplicates(subset=['SK_ID_CURR'])

# Menggabungkan dataset bureau dan bureau_balance berdasarkan kolom 'SK_ID_BUREAU' untuk memasukkan kolom 'SK_ID_BUREAU'
bureau_balance = pd.merge(bureau_balance, bureau[['SK_ID_CURR','SK_ID_BUREAU']], on='SK_ID_BUREAU', how='inner')
# Menghapus nilai yang sama pada kolom 'SK_ID_CURR'
bureau_balance = bureau_balance.drop_duplicates(subset=['SK_ID_CURR'])


In [13]:
# Menggabungkan dataset POS_CASH_balance ke dalam dataset data_inf
merged_data = pd.merge(data_inf, POS_CASH_balance, on='SK_ID_CURR', how='left', suffixes=('_app', '_pos'))

# Menggabungkan dataset bureau_balance ke dalam dataset yang telah digabungkan sebelumnya
merged_data = pd.merge(merged_data, bureau_balance, on='SK_ID_CURR', how='left', suffixes=('', '_bureau'))

# Menggabungkan dataset previous_application ke dalam dataset yang telah digabungkan sebelumnya
merged_data = pd.merge(merged_data, previous_application, on='SK_ID_CURR', how='left', suffixes=('', '_prev'))

# Menggabungkan dataset installments_payments ke dalam dataset yang telah digabungkan sebelumnya
merged_data = pd.merge(merged_data, installments_payments, on='SK_ID_CURR', how='left', suffixes=('', '_installments'))

# Menggabungkan dataset credit_card_balance ke dalam dataset yang telah digabungkan sebelumnya
merged_data = pd.merge(merged_data, credit_card_balance, on='SK_ID_CURR', how='left', suffixes=('', '_credit'))

# Menggabungkan dataset bureau ke dalam dataset yang telah digabungkan sebelumnya
merged_data = pd.merge(merged_data, bureau, on='SK_ID_CURR', how='left', suffixes=('', '_bureau_main'))

data_inf_merged = merged_data

# Menampilkan hasil akhir
data_inf_merged.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY_bureau_main
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,-553.0,NaN,0.0,112500.0,0.0,0.0,0.0,Consumer credit,-155.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,-123.0,NaN,0.0,58500.0,0.0,0.0,0.0,Credit card,-121.0,0.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,-1334.0,19305.0,0.0,688500.0,NaN,NaN,0.0,Car loan,-1334.0,NaN


In [24]:
df_inf_copy = data_inf_merged.copy()

In [25]:
df_inf_copy["SK_ID_PREV_bureau_main"] = 0 

In [26]:
# Removing unnecessary features
df_inf_final = df_inf_copy.drop(Drop_Columns,axis=1).sort_index()
df_inf_final.head(3)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_CREDIT,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_BUREAU_bureau_main,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_SUM,DAYS_CREDIT_UPDATE
0,F,N,Y,0,568800.0,450000.0,Unaccompanied,Working,Higher education,Married,...,3951.0,3951.0,5896630.0,Closed,-857.0,0.0,-492.0,-553.0,112500.0,-155.0
1,M,N,Y,0,222768.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,...,4813.2,4813.2,6735200.0,Closed,-373.0,0.0,-128.0,-123.0,58500.0,-121.0
2,M,Y,Y,0,663264.0,630000.0,NaN,Working,Higher education,Married,...,7875.0,7875.0,5922083.0,Closed,-2005.0,0.0,-909.0,-1334.0,688500.0,-1334.0


In [28]:
# Check Missing Values in Data Inference

df_inf_final.isnull().sum()

CODE_GENDER                0
FLAG_OWN_CAR               0
FLAG_OWN_REALTY            0
CNT_CHILDREN               0
AMT_CREDIT                 0
                       ...  
CREDIT_DAY_OVERDUE      6424
DAYS_CREDIT_ENDDATE     7471
DAYS_ENDDATE_FACT      19586
AMT_CREDIT_SUM          6426
DAYS_CREDIT_UPDATE      6424
Length: 82, dtype: int64

In [29]:
# Imputasi missing values dengan median pada data numerikal
# Imputasi missing values dengan modus pada data kategorikal

category_columns = df_inf_final.select_dtypes(include=['object']).columns.tolist()
integer_columns = df_inf_final.select_dtypes(include=['int64','float64']).columns.tolist()

for column in df_inf_final:
    if df_inf_final[column].isnull().any():
        if(column in category_columns):
            df_inf_final[column]=df_inf_final[column].fillna(df_inf_final[column].mode()[0])
        else:
            df_inf_final[column]=df_inf_final[column].fillna(df_inf_final[column].median())

In [30]:
df_inf_final.isnull().sum()

CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_CREDIT             0
                      ..
CREDIT_DAY_OVERDUE     0
DAYS_CREDIT_ENDDATE    0
DAYS_ENDDATE_FACT      0
AMT_CREDIT_SUM         0
DAYS_CREDIT_UPDATE     0
Length: 82, dtype: int64

In [31]:
# Transform Inference-Set

data_inf_transform = final_pipeline.fit_transform(df_inf_final)
data_inf_transform

c:\Users\SatriyaFzn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 0.92211841, -1.59855663,  0.20295203, ...,  0.        ,
         1.        ,  0.        ],
       [-1.06671236, -0.38292895,  0.64944649, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.56321939,  0.40852841, -0.85608856, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.29557613, -1.04644108,  0.67158672, ...,  0.        ,
         1.        ,  0.        ],
       [-0.45150943,  0.3394647 ,  0.17250923, ...,  0.        ,
         0.        ,  0.        ],
       [-0.40821957, -1.09791369,  0.06918819, ...,  0.        ,
         1.        ,  0.        ]])

In [32]:
predictions = best_xgb.predict(data_inf_transform)

ValueError: Feature shape mismatch, expected: 51, got 49